# Sales Forecasting

Forecast monthly retail sales using ARIMA and Prophet.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Load dataset
# df = pd.read_csv('data/monthly_sales.csv', parse_dates=['date'])
# For demo, generate sample monthly sales
rng = pd.date_range(start='2018-01-01', periods=84, freq='M')
np.random.seed(42)
sales = 20000 + np.linspace(0, 8000, len(rng)) + 3000*np.sin(2 * np.pi * rng.month/12) + np.random.normal(0, 2000, len(rng))
df = pd.DataFrame({'date': rng, 'sales': sales})
df.head()


In [ ]:
# Preprocess

df = df.set_index('date').asfreq('M')

df['sales'].plot(title='Monthly Sales (sample)')
plt.show()

# Train/test split
train = df.iloc[:-12]
test = df.iloc[-12:]

# ARIMA example (using statsmodels)
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(train['sales'], order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit(disp=False)

pred = model_fit.get_forecast(steps=12)
pred_mean = pred.predicted_mean

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae_arima = mean_absolute_error(test['sales'], pred_mean)
rmse_arima = mean_squared_error(test['sales'], pred_mean, squared=False)
print('ARIMA MAE:', round(mae_arima,2))
print('ARIMA RMSE:', round(rmse_arima,2))


In [ ]:
# Optional: Prophet (uncomment to use)
# from prophet import Prophet
# prophet_df = train.reset_index().rename(columns={'date':'ds','sales':'y'})
# m = Prophet(yearly_seasonality=True)
# m.fit(prophet_df)
# future = m.make_future_dataframe(periods=12, freq='M')
# forecast = m.predict(future)
# forecast_tail = forecast.set_index('ds')['yhat'][-12:]
# mae_prophet = mean_absolute_error(test['sales'], forecast_tail)
# print('Prophet MAE:', mae_prophet)


In [ ]:
# Plot results and save a dashboard-like PNG

plt.figure(figsize=(10,5))
plt.plot(train.index, train['sales'], label='Train')
plt.plot(test.index, test['sales'], label='Actual')
plt.plot(pred_mean.index, pred_mean, label='ARIMA Forecast')
plt.title('Monthly Sales Forecast')
plt.legend()
plt.tight_layout()
plt.savefig('dashboard_sales_forecast.png')
plt.show()
